![hopsworks_logo](../images/hopsworks_logo.png)

# Part 02: Training Data & Feature views

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/fraud_batch/2_feature_view_creation.ipynb)

**Note**: you may get an error when installing hopsworks on Colab, and it is safe to ignore it.

This is the second part of the quick start series of tutorials about Hopsworks Feature Store. This notebook explains how to read from a feature group and create training dataset within the feature store

## 🗒️ In this notebook we will see how to create a training dataset from the feature groups:
1. **Select the features** we want to train our model on,
2. **How the features should be preprocessed,**
3. **Create a dataset split** for training and validation data.

![02_training-dataset](../../images/02_training-dataset.png)

#### <span style="color:#ff5f27;">📝 Importing Libraries</span>

In [2]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/164
Connected. Call `.close()` to terminate connection gracefully.


### <span style="color:#ff5f27;">⬇️ Data retrieving from Feature Groups</span>

We start by selecting all the features we want to include for model training/inference.

In [3]:
# Load feature groups.
fares_fg = fs.get_or_create_feature_group("fares_fg",
                                          version=1)
rides_fg = fs.get_or_create_feature_group("rides_fg",
                                          version=1)

In [4]:
# Select features for training data.
fg_query = fares_fg.select(['total_fare'])\
                            .join(rides_fg.select_except(['taxi_id', "driver_id", "pickup_datetime",
                                                          "pickup_longitude", "pickup_latitude",
                                                          "dropoff_longitude", "dropoff_latitude"]),
                                  on=['ride_id'])

fg_query.show(2)

2022-08-21 23:10:08,893 INFO: USE `romankah_featurestore`
2022-08-21 23:10:09,879 INFO: SELECT `fg1`.`total_fare` `total_fare`, `fg0`.`ride_id` `ride_id`, `fg0`.`passenger_count` `passenger_count`, `fg0`.`distance` `distance`, `fg0`.`pickup_distance_to_jfk` `pickup_distance_to_jfk`, `fg0`.`dropoff_distance_to_jfk` `dropoff_distance_to_jfk`, `fg0`.`pickup_distance_to_ewr` `pickup_distance_to_ewr`, `fg0`.`dropoff_distance_to_ewr` `dropoff_distance_to_ewr`, `fg0`.`pickup_distance_to_lgr` `pickup_distance_to_lgr`, `fg0`.`dropoff_distance_to_lgr` `dropoff_distance_to_lgr`, `fg0`.`year` `year`, `fg0`.`weekday` `weekday`, `fg0`.`hour` `hour`
FROM `romankah_featurestore`.`fares_fg_1` `fg1`
INNER JOIN `romankah_featurestore`.`rides_fg_1` `fg0` ON `fg1`.`ride_id` = `fg0`.`ride_id`


,total_fare,ride_id,passenger_count,distance,pickup_distance_to_jfk,dropoff_distance_to_jfk,pickup_distance_to_ewr,dropoff_distance_to_ewr,pickup_distance_to_lgr,dropoff_distance_to_lgr,year,weekday,hour
0,122.0,0e9056d5cea147b4f6fee1a3fb2a8faa,1,68.512120,27.614331,79.633601,24.732814,90.862989,17.100984,75.484501,1970,0,14
1,102.0,b69a847a4f5cf63367f303bf40b69a7e,1,64.445228,33.577944,32.516658,14.633532,53.387452,25.786898,38.673034,1970,0,12


#### <span style="color:#ff5f27;">〰️ Transformation Functions</span>


We can preprocess our data using several encoding methods like *min-max scaling* on numerical features and *label encoding* on categorical features. To do this we simply define a mapping between our features and transformation functions. This ensures that transformation functions such as *min-max scaling* are fitted only on the training data (and not the validation/test data), which ensures that there is no data leakage.

In [5]:
[t_func.name for t_func in fs.get_transformation_functions()]

['standard_scaler', 'label_encoder', 'robust_scaler', 'min_max_scaler']

In [6]:
# # Load transformation functions.
# min_max_scaler = fs.get_transformation_function(name="min_max_scaler")
# label_encoder = fs.get_transformation_function(name="label_encoder")

# # Map features to transformations.
# transformation_functions = {
#     "total_fare": min_max_scaler,
#     "distance": min_max_scaler
# }

## <span style="color:#ff5f27;">🔮 Feature View Creation</span>


We start by selecting all the features we want to include for model training/inference.

After we have made a query from desired features, we should make a corresponding `Feature View`.
In order to do it we may use `fs.create_feature_view()`

In [7]:
nyc_fares_fv = fs.create_feature_view(
    name='nyc_taxi_fares_fv',
    query=fg_query,
    labels=["total_fare"]
#     transformation_functions=transformation_functions
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/164/fs/106/fv/nyc_taxi_fares_fv/version/1


In [8]:
nyc_fares_fv.version

1

## <span style="color:#ff5f27;">🏋️ Training Dataset Creation</span>
    
In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

Training Dataset may contain splits such as:

    Training set - the subset of training data used to train a model.
    Validation set - the subset of training data used to evaluate hparams when training a model
    Test set - the holdout subset of training data used to evaluate a mode

Training dataset is created using fs.create_train_validation_test_split() method.

In [9]:
nyc_fares_fv.create_training_data(
    description='training_dataset',
    data_format='csv'
)

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/164/jobs/named/nyc_taxi_fares_fv_1_1_create_fv_td_21082022211040/executions


(1, <hsfs.core.job.Job at 0x1c38a428a00>)

In [10]:
X_train, y_train = nyc_fares_fv.get_training_data(
    training_dataset_version=1
)

In [11]:
X_train.head(5)

,ride_id,passenger_count,distance,pickup_distance_to_jfk,dropoff_distance_to_jfk,pickup_distance_to_ewr,dropoff_distance_to_ewr,pickup_distance_to_lgr,dropoff_distance_to_lgr,year,weekday,hour
0,9a23189b1efe36f8662715b49042c72e,3,71.630081,58.686481,67.747627,71.669678,57.183055,55.549322,57.118333,1970,0,13
1,1f47756699a6feda1caa858a3a7a799e,1,6.458447,27.240979,23.715037,48.275366,44.421966,34.944513,32.704188,1970,0,13
2,feaf75a34212779be64e72807106047c,4,70.413217,80.828902,32.221322,76.930771,48.827386,70.881912,31.976693,1970,0,13
3,58f08488a117ae8509e93a36f9b6db6e,2,90.428297,32.335778,75.554844,52.641270,65.297882,37.015445,64.926193,1970,0,12
4,dc95dbd0ed7e466f32ba10e6ba7b7646,4,68.531282,20.894081,68.742695,41.915213,73.918481,28.241925,61.435707,1970,0,13


In [12]:
y_train.head(5)

,total_fare
0,132.0
1,168.0
2,163.0
3,50.0
4,196.0


In [13]:
nyc_fares_fv.create_train_test_split(
    test_size=0.2 # here we can define the test dataset size
)

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/164/jobs/named/nyc_taxi_fares_fv_1_2_create_fv_td_21082022211134/executions


(2, <hsfs.core.job.Job at 0x1c38a4783a0>)

In [14]:
X_train, y_train, X_test, y_test = nyc_fares_fv.get_train_test_split(
    training_dataset_version=2
)

In [15]:
X_test.head(5)

,ride_id,passenger_count,distance,pickup_distance_to_jfk,dropoff_distance_to_jfk,pickup_distance_to_ewr,dropoff_distance_to_ewr,pickup_distance_to_lgr,dropoff_distance_to_lgr,year,weekday,hour
0,360ca1b95ffb599317fc2661dffbd738,4,15.969800,62.513030,47.650116,66.117144,54.253071,54.511542,40.677974,1970,0,13
1,8b38223ce53b1ff79764ef29e3d5fa2a,4,25.638115,3.518199,26.699310,17.973022,17.874616,10.570727,16.372117,1970,0,14
2,4baae1dcdceeed7ebe36a0e83f1a4644,4,59.008225,20.619042,71.087304,23.226716,82.173647,10.725451,66.800996,1970,0,14
3,06b9c20527855dc4e9c2681ca8f2a59a,2,42.637934,16.776281,25.952279,4.714794,46.988505,14.468932,33.660930,1970,0,15
4,99cbbd51245e78d3d79c0698bd484e1e,3,65.026397,15.874863,69.719284,7.201506,58.686200,15.513387,59.095903,1970,0,14


In [16]:
y_test.head(5)

,total_fare
0,10.0
1,15.0
2,22.0
3,23.0
4,30.0


## <span style="color:#ff5f27;">⏭️ **Next:** Part 03 </span>

In the next notebook, we will train a model on the dataset we created in this notebook.